<a href="https://colab.research.google.com/github/dassatabdi24/Humor-level-detection/blob/main/Bert_FastText_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import time
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertModel, BertTokenizer, BertForSequenceClassification
import os
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
import json, time, datetime, pickle, random, copy, torch, zipfile, re, string, math
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score
import pickle

In [ ]:
CONFIG = {
    'pair_max_len': 160,
    'single_max_len': 96
}

In [ ]:
df= pd.read_csv( '/content/gdrive/MyDrive/data/task1/train.csv')
print(df)
print(df.loc[df['meanGrade'] == 2.0])

In [ ]:
print(df.iloc[103,1])
print(df.iloc[103,2])
print(df.iloc[103,4])

In [ ]:
class Read_data:

    def __init__(self, path):
        self.path = path

    def replace_edit(self, o, e, pair=True):
        assert len(o) == len(e)
        s_o, s_e = [], []
        for i in range(len(o)):
            s_o.append(re.sub('<(.+)/>', '\\g<1>', o[i]))
            s_e.append(re.sub('<.+/>', e[i], o[i]))
        if pair:
            return list(zip(s_o, s_e))
        return s_e

    def A_helper(self, df, pair=True):
        sents = self.replace_edit(df['original'], df['edit'], pair=pair)
        labels = df['meanGrade'].tolist()
        return (sents, labels)

    def reader(self, pair=True):
        return self.A_helper(pd.read_csv(self.path), pair=pair)
 

In [ ]:

def get_pair_ids(sents, tokenizer, max_len=300, pair=True):
    input_ids, attention_masks, type_ids = [], [], []
    for e in sents:
        if pair:
            t_e = tokenizer.encode_plus(text=(e[0]), text_pair=(e[1]),
            max_length=max_len,
          add_special_tokens=True,
          pad_to_max_length='right')
        else:
            t_e = tokenizer.encode_plus(text=e, 
                                        max_length=max_len,
                                        add_special_tokens=True,
                                        pad_to_max_length='right')
        input_ids.append(t_e['input_ids'])
        attention_masks.append(t_e['attention_mask'])
        type_ids.append(t_e['token_type_ids'])

    return (input_ids, attention_masks, type_ids)


def rmse_A(label, pred):
    rmse = np.sqrt(np.mean((np.array(label) - np.array(pred)) ** 2))
    return rmse

'''
def to_csv(path, df, pred):
    output = pd.DataFrame({'id':df['id'],  'pred':pred})
    output.to_csv(path, index=False)
    with zipfile.ZipFile(path[:-4] + '.zip', 'w') as zf:
        zf.write(path)
'''

def make_even(num):
    arr = np.linspace(0, 3, 16)
    diff, ans = (3, -1)
    for e in arr:
        if abs(e - num) < diff:
            diff = abs(e - num)
            ans = e

    return round(ans, 1)

def format_time(elapsed):
    """
    Takes a time in seconds and returns a string hh:mm:ss
    """
    elapsed_rounded = int(round(elapsed))
    return str(datetime.timedelta(seconds=elapsed_rounded))


def setup_seed(seed):
    CUDA = torch.cuda.is_available()
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if CUDA:
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True


def gpu_info():
    if torch.cuda.device_count() > 0:
        for i in range(torch.cuda.device_count()):
            print('cuda {}: {}'.format(i, torch.cuda.get_device_name(i)))
           

In [ ]:
class My_BERT:

    def __init__(self, pair=True, model_name='bert-base-cased', max_len=256):
        self.device = torch.device(
            'cuda:0' if torch.cuda.is_available() else 'cpu')
        self.CONFIG = {
            'A_train_p': '/content/gdrive/MyDrive/data/task1/train.csv', 
            'A_dev_p': '/content/gdrive/MyDrive/data/task1/dev.csv',
            'A_test_p': '/content/gdrive/MyDrive/data/task1/truth_task_1.csv'
        }
        self.pair = pair
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertForSequenceClassification.from_pretrained(
            model_name, num_labels=1).to(self.device)
        self.max_len = max_len
    def fineT(self, epochs=1, bs=16, verbose=False):

        if verbose:
            print('you are now using {}.'.format(
            torch.cuda.get_device_name(self.device)))

        # read dataset
        
        a_t_s, a_t_l = Read_data(
                self.CONFIG['A_train_p']).reader(pair=self.pair)
        a_v_s, a_v_l = Read_data(
                self.CONFIG['A_dev_p']).reader( pair=self.pair)
        s, l = a_t_s + a_v_s, a_t_l + a_v_l

        #print(s)
        #print(l)
        #tokenization
        t_ids, t_masks, t_types = (torch.tensor(x)
                           for x in get_pair_ids(s, self.tokenizer, max_len=self.max_len, pair=self.pair))
        # create dataloader

        train_data = TensorDataset(t_ids, t_masks, t_types, torch.tensor(l))
        train_dataloader = DataLoader(train_data, shuffle=True, batch_size=bs)

        # optimizer initial
        optimizer = AdamW(self.model.parameters(), lr=2e-05, eps=1e-08)
        total_steps = len(train_dataloader) * epochs
        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=0, num_training_steps=total_steps)

        # start training
        loss_values = []
        self.model.zero_grad()
        for epoch_i in range(epochs):
            if verbose:
                print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))

            t0 = time.time()
            total_loss = 0
            self.model.train()

            for step, batch in enumerate(train_dataloader):
                if step % 100 == 0 and not step == 0:
                    elapsed = format_time(time.time() - t0)
                    if verbose:
                        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(
                            step, len(train_dataloader), elapsed))

                b_input_ids, b_input_mask, b_types, b_labels = tuple(
                    (t.to(self.device) for t in batch))
                outputs = self.model(input_ids=b_input_ids, token_type_ids=b_types,
                                attention_mask=b_input_mask, labels=b_labels)
                loss = outputs[0]
                total_loss += loss.item()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                optimizer.step()
                scheduler.step()
                self.model.zero_grad()

            avg_train_loss = total_loss / len(train_dataloader)
            loss_values.append(avg_train_loss)
            if verbose:
                print('  Average training loss: {0:.2f}'.format(
                    avg_train_loss))
                print('  Training epcoh took: {:}'.format(
                    format_time(time.time() - t0)))

        if verbose:
            print('Training Finished.')

        grand=self.model.state_dict()
        
        file_name = '/gdrive/MyDrive/data/pretrainedc/{2}/Bert_{2}_regress_{0}epochs_{1}bs.pt'.format(
            epochs, bs, 'pair' if self.pair else 'single')

        torch.save(self.model.state_dict(), os.getcwd() + file_name)

        print(file_name + ' saved.')
        torch.cuda.empty_cache()
        return self

    def predict(self, pretrained=None, task='A', bs=128):
        test_s, test_l = Read_data(
            self.CONFIG['A_test_p']).reader( pair=self.pair)


        # create dataloader
 
        ids, masks, types = (torch.tensor(x) for x in get_pair_ids(
                test_s, self.tokenizer, max_len=self.max_len, pair=self.pair))
        
        data = TensorDataset(ids, masks, types)
        dataloader = DataLoader(data, batch_size=bs)

        tag = 'pair' if self.pair else 'single'
        
        if pretrained:
            self.model.load_state_dict(torch.load(
                    '{0}/gdrive/MyDrive/data/pretrained/{1}/{2}'.format(os.getcwd(), tag, pretrained)))
        self.model.eval()

        pred = []
        for step, batch in enumerate(dataloader):
            b_input_ids, b_input_mask, b_types = tuple(
                (t.to(self.device) for t in batch))
            with torch.no_grad():
                outputs = self.model(
                    input_ids=b_input_ids, token_type_ids=b_types, attention_mask=b_input_mask)
            logits = outputs[0]
            #print(len(logits))
            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            pred.extend(logits)
        
        return pred



    def evaluate(self, pretrained=None, task='A',  bs=128):
        pred = [make_even(x) for x in self.predict(pretrained=pretrained, task=task, bs=bs)]
        test_s, test_l = Read_data(
            self.CONFIG['A_test_p']).reader( pair=self.pair)
        
        res =rmse_A(test_l, pred)
        print('{} model task A RMSE = {:.5f}'.format(
                pretrained[:-3] if pretrained else 'this model', res))
        print(res)
        return round(res, 5)

In [ ]:
def get_res(tag, pair=True, max_len=CONFIG['pair_max_len'], task='A'):
    # for pair regress
    if not os.path.exists(os.getcwd() +'/gdrive/MyDrive/data/pretrained/'):
       print('no pretrained model files, please train first!')
    pretrains = os.listdir(os.getcwd()+'/gdrive/MyDrive/data/pretrained/' + tag)
    model = My_BERT(pair=pair, max_len=max_len)
    res = [model.evaluate(pretrained=pretrains[i], task=task) for i in range(0,len(pretrains))]
    del model
    return res

def get_A():
    return {
        'pair_regress': get_res('pair', pair=True, max_len=CONFIG['pair_max_len'], task='A'),
        #'single_regress': get_res('single', pair=False, max_len=CONFIG['single_max_len'], task='A'),
        
    }

    

def save_res():
    res = {
        'A': get_A(),
        
    }
    with open('results.pickle', 'wb') as f:
        pickle.dump(res, f, protocol=pickle.HIGHEST_PROTOCOL)
    return res



def train_bert(pair=False, model='bert-base-cased', max_len=CONFIG['single_max_len']):
    My_BERT(pair=pair, model_name=model,max_len=max_len).fineT(epochs=1,bs=32,verbose=True)

def train():
    # training part, training file will be saved in pretrained folder
    # for bert pair regressor training
    train_bert(pair=True, max_len=CONFIG['pair_max_len'])
    # for bert single regressor training
    #train_bert(pair=False, max_len=CONFIG['single_max_len'])
   


In [ ]:
train()

In [ ]:
save_res()

In [ ]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle

In [ ]:
import tensorflow as tf 
import tensorflow_hub as hub 
import tensorflow.compat.v1 as tf1
tf1.disable_eager_execution()
tf1.disable_v2_behavior()

In [ ]:
CONFIG = {
            'A_train_p': '/content/gdrive/MyDrive/data/task1/train.csv',
            'A_dev_p': '/content/gdrive/MyDrive/data/task1/dev.csv',
            'A_test_p': '/content/gdrive/MyDrive/data/task1/truth_task_1.csv',
            'B_train_p': '/content/gdrive/MyDrive/data/task2/train.csv',
            'B_dev_p': '/content/gdrive/MyDrive/data/task2/dev.csv',
            'B_test_p': '/content/gdrive/MyDrive/data/task2/truth_task_2.csv',
        }

In [ ]:
def get_data():
        t_s, t_l = Read_data(
            CONFIG['A_train_p']).reader(pair=False)
        v_s, v_l = Read_data(
            CONFIG['A_dev_p']).reader(pair=False)
        tv_s, tv_l = t_s + v_s, t_l + v_l

        test_s, test_l = Read_data(CONFIG['A_test_p']).reader(
             pair=False)

       
        return (tv_s, tv_l, test_s, test_l)

In [ ]:
tv_f,tv_l,test_s,test_l=get_data()
print(len(tv_f))

In [ ]:
from gensim.models import FastText
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

corpus= tv_f+test_s
docs = [d.lower().split() for d in corpus]
#print(docs)
# train fasttext from gensim api

ft = FastText(size=100, window=2, min_count=1, seed=33)
ft.build_vocab(docs)
ft.train(docs, total_examples=ft.corpus_count, epochs=10)

# prepare text for keras neural network

max_len = 96
#fitting tokenizer to doc
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=True)
tokenizer.fit_on_texts(docs)
'''
sequence_docs = tokenizer.texts_to_sequences(docs)
sequence_docs = tf.keras.preprocessing.sequence.pad_sequences(sequence_docs, maxlen=max_len,padding='post')
#print(sequence_docs[0])
#print(len(sequence_docs[0]))
# extract fasttext learned embedding and put them in a numpy array
'''
embedding_matrix_ft = np.random.random((len(tokenizer.word_index) + 1, ft.vector_size))

pas = 0
for word,i in tokenizer.word_index.items():
    
    try:
        embedding_matrix_ft[i] = ft.wv[word]
    except:
        pas+=1
print(type(embedding_matrix_ft))
print(embedding_matrix_ft.shape)
print(ft.vector_size)
# define a keras model and load the pretrained fasttext weights matrix

In [ ]:
sequence_docs_train = tokenizer.texts_to_sequences(tv_f)
sequence_docs_train = tf.keras.preprocessing.sequence.pad_sequences(sequence_docs_train, maxlen=max_len,padding='post')
sequence_docs_test = tokenizer.texts_to_sequences(test_s)
sequence_docs_test = tf.keras.preprocessing.sequence.pad_sequences(sequence_docs_test, maxlen=max_len,padding='post')

model=Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, ft.vector_size, 
                weights=[embedding_matrix_ft],input_length=max_len, trainable=True))

model.add(LSTM(6))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(sequence_docs_train,tv_l,epochs=4,verbose=1,batch_size=32)

In [ ]:
preds=model.predict(sequence_docs_test)
print(len(preds))
print(preds)

In [ ]:
print(rmse_A(test_l,preds.tolist()))

In [ ]:
class Bert_fasttext_LSTM:
    def __init__(self, preds, task='A', best_bert='Bert_pair_regress_1epochs_32bs.pt', bert_weight=0.91, fasttextlstm_weight=0.09):
        self.task = task
        self.CONFIG = {
            'pair_max_len': 160,
            'single_max_len': 96,  # max length for train, dev, test dataset is 74
            'A_test_p': '/content/gdrive/MyDrive/data/task1/truth_task_1.csv',
            'B_test_p': '/content/gdrive/MyDrive/data/task2/truth_task_2.csv',
        }
        self.best_bert = best_bert
        self.pair = True if 'pair' in best_bert else False
        self.bert_weight = bert_weight
        self.fasttextlstm_weight = fasttextlstm_weight
        self.fasttext_lstm_pred= preds

    def predict(self):

        bert_pred = My_BERT(pair=self.pair,
                                 max_len=self.CONFIG['pair_max_len'] ).predict(
                                     task=self.task, pretrained=self.best_bert)
        
        print(len(bert_pred))
        print(len(self.fasttext_lstm_pred))
        pred = [make_even(self.bert_weight*x + (np.asarray(self.fasttextlstm_weight)*y).tolist())
                for (x, y) in zip(bert_pred, self.fasttext_lstm_pred)]
        return pred
    
    def evaluate(self):
        pred = self.predict()
        test_s, test_l = Read_data(
            self.CONFIG['A_test_p']).reader( pair=self.pair)
        res = rmse_A(test_l, pred)
        
        print('{0}*{1} + {2}*fasttext_LSTM model task A RMSE = {3:.5f}'.format(
            self.bert_weight, self.best_bert[:-3], self.fasttextlstm_weight, res))
        return round(res, 5)

In [ ]:
Bert_fasttext_LSTM(preds.tolist(),task='A',
             best_bert='Bert_pair_regress_1epochs_32bs.pt').evaluate()

In [ ]:


class NB_LSTM:

    def __init__(self, task='A'):
        self.CONFIG = {
            'A_train_p': '/content/gdrive/MyDrive/data/task1/train.csv',
            'A_dev_p': '/content/gdrive/MyDrive/data/task1/dev.csv',
            'A_test_p': '/content/gdrive/MyDrive/data/task1/truth_task_1.csv',
            'B_train_p': '/content/gdrive/MyDrive/data/task2/train.csv',
            'B_dev_p': '/content/gdrive/MyDrive/data/task2/dev.csv',
            'B_test_p': '/content/gdrive/MyDrive/data/task2/truth_task_2.csv',
        }
        self.task = task
        self.retok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
        self.model = Sequential()
        
        self.tv_f = None
        self.tv_l = None
        self.test_f = None
        self.test_l = None

    def get_data(self):
        t_s, t_l = Read_data(
            self.CONFIG['A_train_p']).reader( pair=False)
        v_s, v_l = Read_data(
            self.CONFIG['A_dev_p']).reader( pair=False)
        tv_s, tv_l = t_s + v_s, t_l + v_l

        test_s, test_l = Read_data(self.CONFIG['A_test_p']).reader(
             pair=False)

        return (tv_s, tv_l, test_s, test_l)

    def tokenize(self, s):
        return self.retok.sub(r' \1 ', s).split()

    def pr(self, trn_term_doc, y_i, y):
        p = trn_term_doc[(y == y_i)].sum(0)
        return (p + 1) / ((y == y_i).sum() + 1)

    def get_features(self):
        tv_s, tv_l, test_s, test_l = self.get_data()
        vec = TfidfVectorizer(ngram_range=(1, 2), tokenizer=(self.tokenize), min_df=3,
                              max_df=0.9,
                              strip_accents='unicode',
                              use_idf=1,
                              smooth_idf=1,
                              sublinear_tf=1)
        trn_term_doc = vec.fit_transform(tv_s) #sparse matrix
        test_term_doc = vec.transform(test_s) #sparse matrix
        print(type(trn_term_doc))
        print(type(test_term_doc))
      
        r = np.log(self.pr(trn_term_doc, 1, np.array(tv_l)) /
                   self.pr(trn_term_doc, 0, np.array(tv_l)))
        tv_f = trn_term_doc.multiply(r)
        test_f = test_term_doc.multiply(r)
        
        
        tv_f=tv_f.toarray()
        test_f=test_f.toarray()
        
        return (tv_f, tv_l, test_f, test_l)

    def train(self):
        self.tv_f, self.tv_l, self.test_f, self.test_l = self.get_features()
        print(self.tv_f.shape)
        self.tv_l = np.array(self.tv_l)
        self.tv_f = self.tv_f[:, :, None] 
        print(self.tv_f.shape)
        print(self.tv_f.shape[1:])
        
        self.model.add(LSTM(units=1,input_shape = self.tv_f.shape[1:]))
        #self.model.add(LSTM(units=10))
        self.model.add(Dense(1))
        self.model.compile(loss='mean_squared_error', optimizer='adam')
        
        self.model.fit(self.tv_f, self.tv_l,epochs=1,verbose=1,batch_size=128)
        
        return self
    
    def predict(self):
    
        #self.test_f = self.test_f[:, :, None]
        return self.model.predict(self.test_f)


    def evalute(self):
        if self.task == 'A':
            preds=self.predict()
            print(preds.shape)
            print(preds)
            preds=preds.reshape((preds.shape[0],))    
            preds=preds.tolist()
            print(preds)
            print(self.test_l)
            res = rmse_A(self.test_l, self.predict())
            print('NB-LSTM model task A RMSE: %.5f' % res)
        else:
            res = accuracy_score(self.test_l, to_label(self.predict(), len(self.test_l)))
            print('NB-LSTM model task B accuracy: %.5f' % res)
        return round(res, 5)
        

In [ ]:

n=NB_LSTM('A')
n.train()
n.test_f=n.test_f[:,:,None]
preds1=n.model.predict(n.test_f)
preds_lstm= preds1.tolist()
print(rmse_A(n.test_l,preds_lstm))
